# Netzsimulation mit Pandapower und Simbench

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandapower as pp
import pandas as pd
import simbench as sb
from pandapower.timeseries import OutputWriter, run_timeseries
from pandapower.control import ConstControl

from src.load_profiles import get_load_profile

In [ ]:
# Tipp um eigene Funktionen automatisch neu zu laden
%load_ext autoreload
%autoreload 2

## Pandapower Basics

Pandapower ist eine Bibliothek zur Berechnung/Simulation von Stromnetzen.
Es verwendet pandas und [PYPOWER](https://github.com/rwl/PYPOWER) um Netzwerkberechnungen zu ermöglichen.

* Website: http://www.pandapower.org/
* Dokumentation: https://pandapower.readthedocs.io/en/latest/about.html.
* Tutorials: https://nbviewer.org/github/e2nIEE/pandapower/tree/develop/tutorials/.

Konvention fuer Namen von Einheiten: \<parameter\>\_\<unit\> oder \<parameter\>\_\<phase\>\_\<unit\>. Beispiel: vn_kv -> $v_n[kV]$. Naehere Infos in Function Docstring oder in der Dokumentation.

Als erstes bauen wir ein minimales Netz aus einzelnen Komponenten auf.

In [ ]:
# create empty net
my_net = pp.create_empty_network()

In [ ]:
my_net

In [ ]:
# create buses
bus1 = pp.create_bus(my_net, vn_kv=20.0, name="Bus 1")
bus2 = pp.create_bus(my_net, vn_kv=0.4, name="Bus 2")
bus3 = pp.create_bus(my_net, vn_kv=0.4, name="Bus 3")

In [ ]:
my_net.bus

In [ ]:
# The returned value is the index of the element in its table
print(type(bus1), bus1)

In [ ]:
# create bus elements
pp.create_ext_grid(
    my_net, bus=bus1, vm_pu=1.02, name="Grid Connection"
)  # erstellt externes netz
pp.create_load(my_net, bus=bus3, p_mw=0.100, q_mvar=0.05, name="Load")  # erstellt last

In [ ]:
# create branch elements
trafo = pp.create_transformer(
    my_net, hv_bus=bus1, lv_bus=bus2, std_type="0.4 MVA 20/0.4 kV", name="Trafo"
)
line = pp.create_line(
    my_net,
    from_bus=bus2,
    to_bus=bus3,
    length_km=0.1,
    std_type="NAYY 4x50 SE",
    name="Line",
)

In [ ]:
my_net

In [ ]:
my_net.ext_grid

In [ ]:
pp.plotting.simple_plot(my_net, plot_loads=True, plot_gens=True, plot_sgens=True)

In [ ]:
pp.runpp(my_net, numba=False)

In [ ]:
my_net.res_bus

In [ ]:
[el for el in dir(my_net) if el.startswith('res')]

In [ ]:
# Unit descriptions in https://pandapower.readthedocs.io/en/latest/elements/line.html#result-parameters
my_net.res_line

## Beispielnetze und Simulation 

### Simples Pandapower Beispielnetz

In [ ]:
# https://pandapower.readthedocs.io/en/latest/networks/test.html#pandapower.networks.simple_four_bus_system
small_net = pp.networks.simple_four_bus_system()
pp.plotting.simple_plot(small_net, plot_loads=True, plot_gens=True, plot_sgens=True)

In [ ]:
small_net

In [ ]:
small_net.load

In [ ]:
small_net.sgen

### Zeitreihenberechnung mit Hilfe von pandapower

Basierend auf dem [Simple Time Series Example](https://github.com/e2nIEE/pandapower/blob/develop/tutorials/time_series.ipynb) Tutorial von pandapower, versuchen wir dieses simple Netz mit unseren vorher definierten Zeitreihen zu koppeln.

In [ ]:
# Step 1 net, already done

In [ ]:
# Step 2, import datasources
pv = pd.read_csv('outputs/pv_example.csv', index_col=0)

In [ ]:
lastprofile_file = (
    Path("data") / "representative_profiles_vdew.xls"
)
assert (
    lastprofile_file.is_file()
), f"Did not find file with representative load profiles at {lastprofile_file}"

In [ ]:
pv.index = pd.to_datetime(pv.index)
pv = pv / 10**3

In [ ]:
# H0 at the moment has non-matching timezone and exploding values in the end of year!
h0profile = get_load_profile(
    lastprofile_file,
    from_=pv.index[0],
    to=pv.index[-1],
    type="H0",
)

In [ ]:
h0profile = h0profile.resample('H').mean() / 10**6 * 5

In [ ]:
df = pd.DataFrame({'PV': pv.squeeze(), 'H0': h0profile.squeeze()})
df.head()

In [ ]:
df.plot()

In [ ]:
small_net.sgen

In [ ]:
# Step 3: Create Controllers, attach to networ
ConstControl(small_net, element='load', variable='p_mw', element_index=[0, 1],
                 data_source=df, profile_name=["H0", "H0"])

In [ ]:
ConstControl(small_net, element='sgen', variable='p_mw', element_index=[0, 1],
                 data_source=df, profile_name=["PV", "PV"])

In [ ]:
# Step 4: Create Output Writer, copy from nb
output_dir = Path('output') / 'small_net'
if output_dir.exists == False:
    output_dir.mkdir()

ow = OutputWriter(small_net, df.index, output_path=output_dir, output_file_type=".json", log_variables=list())
# these variables are saved to the harddisk after / during the time series loop
ow.log_variable('res_load', 'p_mw')
ow.log_variable('res_bus', 'vm_pu')
ow.log_variable('res_line', 'loading_percent')
ow.log_variable('res_line', 'i_ka')

In [ ]:
df.index.to_list()[0]

In [ ]:
# Step 5: run timeseries
run_timeseries(small_net, df.index.to_list(), numba=True)

### Beispielnetz aus Simbench inklusive Beispielzeitreihen

Simbench bietet viele Beispielnetze für unterschiedlichste Szenarien an: https://simbench.de/de/datensaetze/.
(Link funktionierte bei mir nicht mit Firefox).

Dieses Beispiel Notebook gibt einen guten Überblick über die möglichen Nutzungen der Netzwerke gemeinsam mit pandapower:
https://github.com/e2nIEE/simbench/blob/master/tutorials/simbench_grids_basics_and_usage.ipynb.

In [ ]:
# simbench - simbench.de - Zeitreihen von Verbrauchs/Einspeisedaten + Netze.
sb_code_lv = "1-LV-semiurb4--0-sw"

# creating a standard lv test network
net = sb.get_simbench_net(sb_code_lv)

In [ ]:
pp.plotting.simple_plot(net, plot_loads=True, plot_gens=True, plot_sgens=True)

In [ ]:
for key, df in net.profiles.items():
    print(f"{key}: {list(df.columns)}\n")

In [ ]:
# Get PV data from simbench
pv = net.profiles["renewables"].copy()
pv.head()

In [ ]:
# Make time the index, dayfirst for day.month.year and sort_index to allow slicing
pv.index = pd.to_datetime(pv["time"], dayfirst=True)
pv = pv["PV5"].sort_index()
pv.tail()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 3), sharey=True)
fig.suptitle("Simbench PV profile")
pv["2016-01-01":"2016-01-10"].plot(ax=axs[0])
pv["2016-06-01":"2016-06-10"].plot(ax=axs[1])

In [ ]:
h0A = net.profiles["load"].set_index("time")["H0-A_pload"].copy()
h0A.index = pd.to_datetime(h0A.index, dayfirst=True)
h0A = h0A.sort_index()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 3), sharey=True)
fig.suptitle("Simbench H0 profile")
h0A["2016-01-01":"2016-01-10"].plot(ax=axs[0])
h0A["2016-06-01":"2016-06-10"].plot(ax=axs[1])

### Run timeseries analysis with predefined profiles

After [tutorial](https://github.com/e2nIEE/simbench/blob/master/tutorials/simbench_grids_basics_and_usage.ipynb).

In [ ]:
# Check that all profiles exist
assert not sb.profiles_are_missing(net)

In [ ]:
profiles = sb.get_absolute_values(net, profiles_instead_of_study_cases=True)

In [ ]:
# Apply ConstControllers
sb.apply_const_controllers(net, profiles)
# Define time steps for 1 day
time_steps = range(0, 96)

In [ ]:
# Create Output writer
output_dir = Path("./outputs")
output_dir.mkdir(exist_ok=True)
output_dir = output_dir / "simbench_ts_run"
# Using json here, as csv has non standard formatting, could also use .xls
ow = OutputWriter(net, time_steps, output_path=output_dir, output_file_type=".json")
ow.log_variable("res_load", "p_mw", eval_function=np.sum, eval_name="Load Sum")
ow.log_variable("res_bus", "vm_pu", eval_function=np.min, eval_name="min_vm_pu")
ow.log_variable("res_bus", "vm_pu", eval_function=np.max, eval_name="max_vm_pu")

In [ ]:
run_timeseries(net, time_steps, numba=False)

In [ ]:
# Read Results and create DataFrame
vm_pu_file = output_dir / "res_bus" / "vm_pu.json"
vm_pu = pd.read_json(vm_pu_file)
ll_file = output_dir / "res_line" / "loading_percent.json"
line_loading = pd.read_json(ll_file)
load_p_file = output_dir / "res_load" / "p_mw.json"
load_p = pd.read_json(load_p_file)

In [ ]:
# Plot line loading
vm_pu[['min_vm_pu', 'max_vm_pu']].plot()
plt.xlabel("time step")
plt.ylabel("voltage mag. [p.u.]")
plt.title("Voltage Magnitude")
plt.grid()

In [ ]:
# load results
load_p.sort_index().plot()
plt.xlabel("time step")
plt.ylabel("P [MW]")
plt.title("Sum of Loads")
plt.grid()
plt.show()